In [5]:
import aiohttp # pip3 install aiohttp aiodns
import asyncio # pip3 install asyncio
import os
import requests
import pandas as pd
from datetime import datetime
from typing import List

OKX_API = "https://www.okx.com/"
TICKER_ENDPOINT = "api/v5/market/ticker"
TICKERS_ENDPOINT = "api/v5/market/tickers"
INSTRUMENTS_ENDPOINT = "api/v5/public/instruments"

In [6]:
async def get_ticker(
        session: aiohttp.ClientSession,
        inst_id: str,
    ) -> dict:
    """
    @notice return market data for instrument id, ie OHLC / current price / volume
    
    @param inst_id, instrument id for coin, i.e. 'BTC-USD-SWAP' for btc perp
    """
    url = os.path.join(OKX_API, TICKER_ENDPOINT)
    params = {"instId": inst_id.upper()}
    resp = await session.request('GET', url=url, params=params)
    data = await resp.json()
    return data


def get_tickers(
        coin: str, 
        inst_type: str = "FUTURES",
    ) -> List[str]:
    """
    @notice return instrument names of all inst_types associated with coin
    
    @param coin, name of coin to return instruments for
    """
    url = os.path.join(OKX_API, TICKERS_ENDPOINT)
    resp = requests.get(url, params={"instType": inst_type.upper()})
    data = resp.json()
    return [d['instId'] for d in data['data'] if d['instId'].startswith(f'{coin.upper()}-USDT')]


def get_instruments(
        coin: str,
    ) -> List[dict]:
    """
    @notice returns instrument info - including expiration date for futures, 
            which is not included in the /tickers endpoint
    """
    resp = requests.get(os.path.join(OKX_API, INSTRUMENTS_ENDPOINT), {'instType': 'FUTURES'})
    data = resp.json()['data']
    return [d for d in data if d['instFamily'] == f"{coin.upper()}-USD"]


def sort_instruments(
        instruments: list,
    ) -> List[str]:
    """
    @notice this sorts instruments based on string date, i.e.,
        ['BTC-USDT-240426',            ['BTC-USDT-240329',
         'BTC-USDT-240329']    --->     'BTC-USDT-240426']
    """
    term_instruments_ = list(instruments)
    term_split = [i.split('-') for i in term_instruments_]
    term_sorted = [(base, quote, datetime.strptime(date, '%y%m%d')) for base, quote, date in term_split]
    term_sorted.sort(key=lambda x: x[-1])
    
    return [f"{base}-{quote}-{datetime.strftime(date, '%y%m%d')}" for base, quote, date in term_sorted]


async def main(
        instruments: list,
    ) -> List[dict]:
    """
    @notice asynchronously get ticker data for given list of instruments
    """
    
    async with aiohttp.ClientSession() as session:
        tasks = []
        for instrument in instruments:
            tasks.append(get_ticker(session, instrument))

        try:
            data = await asyncio.gather(*tasks, return_exceptions=False)
            return data

        except ValueError as e:
            raise ValueError(e)

In [21]:
coin = 'BTC'
term_instruments = sort_instruments(get_tickers(coin))
perp_instrument = f"{coin.upper()}-USDT-SWAP" # /USD or /USDT perps available
spot_instrument = f"{coin.upper()}-USDT"
instruments = [spot_instrument] + [perp_instrument] + term_instruments
instruments

['BTC-USDT',
 'BTC-USDT-SWAP',
 'BTC-USDT-240517',
 'BTC-USDT-240524',
 'BTC-USDT-240531',
 'BTC-USDT-240628',
 'BTC-USDT-240927',
 'BTC-USDT-241227']

In [22]:
# assign expiries (milliseconds) to term futures
instruments_info = get_instruments(coin)
expiries = {}
for ins in term_instruments:
    for d in instruments_info:
        inst_id = d['instId'].replace('USD', 'USDT')
        if ins == inst_id:
            expiries[ins] = int(d['expTime'])
            break

In [23]:
data = await main(instruments)
data_clean = [d['data'][0] for d in data]
df = pd.DataFrame([
    {
        'instrument_name':d['instId'], 
        'base_ccy': coin.upper(), 
        'expiration_timestamp_ms': expiries.get(d['instId'], None), 
        'mark_price': (float(d['bidPx']) + float(d['askPx'])) / 2.
    } 
    
    for d in data_clean
])
df

,instrument_name,base_ccy,expiration_timestamp_ms,mark_price
0,BTC-USDT,BTC,NaN,61444.95
1,BTC-USDT-SWAP,BTC,NaN,61439.85
2,BTC-USDT-240517,BTC,1.715933e+12,61494.35
3,BTC-USDT-240524,BTC,1.716538e+12,61575.05
4,BTC-USDT-240531,BTC,1.717142e+12,61673.85
5,BTC-USDT-240628,BTC,1.719562e+12,62087.35
6,BTC-USDT-240927,BTC,1.727424e+12,63734.15
7,BTC-USDT-241227,BTC,1.735286e+12,65666.65
